<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Notebooks/03-analisis-univariante-causal-multivariante/03_C___Introducci%C3%B3n_analisis_causal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 INTRO: Análisis Causal 
El análisis causal trata de encontrar las relaciones causales (no solo asociaciones) entre las variables.

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1WNWGnLUyiGqwFf92V3V_szu3QTD_0EQY5GZjhFgen5Q/edit?usp=sharing)
*   [Enlace a este notebook en Colab](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Notebooks/03-analisis-univariante-causal-multivariante/03_C___Introducci%C3%B3n_analisis_causal.ipynb)




## 0101 Qué es un análisis univariante?
El análisis univariante consiste en estudiar la distribución y características de una sola variable.

Veamos con ejemplos:

## 0102 Import
Importamos todas las librerías necesarias para este análisis ([¿No sabes lo que es una librería de Python?](https://www.quora.com/What-is-a-Python-library-and-what-can-I-use-it-for)): pandas, numpy, seaborn, matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 0103 Carga el dataset Fifa19
Para probar a hacer análisis univariantes utilizaremos el dataframe con datos ya limpios del fifa 19 ([aquí](https://drive.google.com/file/d/1P0sQHp1bPiKbL-MSr7FndJRgowvOLdjU/view?usp=sharing) está mi archivo ya limpio). Importa este dataset en un dataframe llamado **df_fifa19**.

Utiliza solo las columnas cols = ['Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club', 
        'Height_cm', 'Weight_kg', 'Value_€', 'Wage_€', 
        'Release Clause_€', 'Preferred Foot', 'Position', 'SprintSpeed'] y en este orden.

In [ ]:
cols = ['Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club', 
        'Height_cm', 'Weight_kg', 'Value_€', 'Wage_€', 
        'Release Clause_€', 'Preferred Foot', 'Position', 'SprintSpeed']

# Url
url = "https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/Fifa19_Clean.csv"

# Importamos el archivos usando read_csv()
df_fifa19 = pd.read_csv(url, usecols=cols)
df_fifa19.head(3)

# 02 Azar
El azar puede ser la fuente de una correlación.

Imaginémonos dos monedas de 1€. Una es de Pepe, y la otra de Carmen. Se ponen de acuerdo y tiran sus respectivas monedas 10 veces. Anotan un 1 si sale cara, un 0 si sale cruz.  

## 0201 Probabilidad de correlación
Qué probabilidad hay de que (solo por azar) obtengan una correlación entre ambas monedas mayor a [0.5]?

In [ ]:
# Definimos una moneda
moneda = pd.Series([0,1])
moneda

In [ ]:
# Ejemplo de tirada
moneda.sample(10, replace=True).reset_index(drop=True)

In [ ]:
#Experimento
# Primero generamos una variable donde guardaremos los valores de la correlación
correlacion = []

# Empezamos experimento 
for n_experimento in range(10000):
  tirada_pepe = moneda.sample(10, replace=True).reset_index(drop=True)
  tirada_carmen = moneda.sample(10, replace=True).reset_index(drop=True)
  correlacion.append(tirada_pepe.corr(tirada_carmen))

In [ ]:
# Visualizamos la distribución de correlación
plt.hist(correlacion, bins=[-1, -0.5, 0, 0.5, 1])
plt.show()

In [ ]:
(np.abs(correlacion)>=0.5).sum() / len(correlacion)

## 0202 Y si tiran la moneda 20 veces? y 100?
La probabilidad de que el azar sea el causante de una correlación disminuye cuando el tamaño de la muestra es mayor (es decir, cuando tiran la moneda más veces).

In [ ]:
# EXPERIMENTO
# Inicializamos un dataframe para guardar los resultados
df_resultados = pd.DataFrame()

# Probamos para distintos números de veces que se tiran la moneda
for n_tiradas in [5, 10, 20, 50, 100]:

  # Primero generamos una variable donde guardaremos los valores de la correlación
  correlacion = []

  # Empezamos experimento 
  for n_experimento in range(10000):
    tirada_pepe = moneda.sample(n_tiradas, replace=True).reset_index(drop=True)
    tirada_carmen = moneda.sample(n_tiradas, replace=True).reset_index(drop=True)
    correlacion.append(tirada_pepe.corr(tirada_carmen))
  
  # Guardamos el resultado en el dataframe
  df_resultados[str(n_tiradas)+'_tiradas'] = correlacion

In [ ]:
df_resultados.head()

In [ ]:
filtro_interes = df_resultados.abs()>=0.5
filtro_interes.sum()/len(df_resultados)

# 03 Relación causal directa
No hay métodos estadísticos que nos permitan saber si la relación entre dos variables es causal. Sólo conocimiento sobre el sistema que genera los datos nos permitirá hacer este tipo de juicios.

Lo que sí podemos calcular es la fortaleza de la asociación entre las variables, como hemos visto en el [notebook de análisis bivariantes](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Notebooks/03-analisis-univariante-causal-multivariante/03_B___Analisis_multivariante.ipynb). Por ejemplo, podríamos utilizar la función desarrollada en el [EJERCICIO 5](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Notebooks/02-analisis-exploratorio-visual/EJERCICIO_5___EDA_Visual_Funcion_Scatter.ipynb).


In [ ]:
def evaluacion_correlacion(vector_x, nombre_x, vector_y, nombre_y):
  # Estilo seaborn
  sns.set()

  # Tamaño de 7.7 x 7.5
  plt.figure(figsize=(7.5, 7.5))

  # DIAGRAMA DE DISPERSIÓN
  # Simplemente utilizamos plt.scatter()
  plt.scatter(vector_x, 
              vector_y, 
              alpha = 0.05)
  
  # COEFICIENTE DE CORRELACIÓN
  # Calculamos el coeficiente pearson redondeando a dos cifras (0.XX) ... 
  # ... y lo guardamos en formato texto (str)
  corr_formato_texto = str(round(np.corrcoef(vector_x, vector_y)[0,1],2))

  # Calculamos las coordenadas donde posicionar el texto (en el medio del gráfico)
  coordenada_x = vector_x.min() + (vector_x.max()-vector_x.min())/2
  coordenada_y = vector_y.min() + (vector_y.max()-vector_y.min())/2
  
  # Utilizamos la función plt.text() 
  plt.text(s = corr_formato_texto, 
           x = coordenada_x, 
           y = coordenada_y,
           fontsize=150, 
           horizontalalignment='center',
           verticalalignment='center', 
           color = 'white', 
           alpha = 0.85)
  
  # Añadimos detalles a la imagen
  plt.ylabel(nombre_y)
  plt.xlabel(nombre_x)
  plt.title('CORRELACIÓN ' + nombre_x + ' VS ' + nombre_y,
            fontsize = 15)
  plt.show()

# 04 Presencia de confounders
La presencia de confounders puede hacer que la asociación que observamos entre dos variables no sea la que corresponde a su relación directa.

## EJERCICIO 1
Cuál es la influencia del nivel fubolístico sobre el valor del futbolista? 

Ver el diagrama causal propuesto:
https://docs.google.com/presentation/d/1jYSOQpCVbL7JdacYhq8NuKWwhYe2QwW8qvM680GERps/edit#slide=id.gbc327d14bd_0_11

La variable edad es un confounder: afecta tanto al nivel futbolistico actual como al valor.

In [ ]:
# Utilizaremos este dataframe
df_valor = df_fifa19[['Name','Value_€','Overall','Age']]
df_valor.head()

Primero muestra la asociación entre las 3 variables con con sns.pairplot() y la visualización de la matriz de los coeficientes de Pearson.

In [ ]:
sns.pairplot(df_valor, kind='scatter', diag_kind='kde', plot_kws={'alpha':0.1})

In [ ]:
# Visualización coeficientes Pearson
plt.figure(figsize=(8,7))
sns.heatmap(np.round(df_valor.corr(),2), vmin=-1, vmax=1, annot=True, cmap="coolwarm")
plt.show()

Vamos a centrarnos ahora en los futbolistas que tienen una misma edad. Por ejemplo, vamos a centrarnos en aquellos que tienen 25 años.

In [ ]:
# Ajustamos por aquellos que tienen un cierto valor
filtro = df_valor['Age'] == 25

Ahora vuelve a mostrar las distribuciones y asociaciones, pero sólo para este grupo de futbolistas.

In [ ]:
sns.pairplot(df_valor.loc[filtro], 
             kind='scatter', 
             diag_kind='kde', 
             plot_kws={'alpha':0.1})

In [ ]:
# Visualización coeficientes Pearson
plt.figure(figsize=(8,7))
sns.heatmap(np.round(df_valor.loc[filtro].corr(),2), 
            vmin=-1, vmax=1, 
            annot=True, cmap="coolwarm")
plt.show()

Genera un loop que evalúe la relación entre el nivel futbolístico y el valor para cada edad (entre 25 y 35 años) utilizando la función evaluacion_correlacion().

In [ ]:
for edad in [25,26,27,28,29,30,31,32,33,34,35]:
  filtro_edad = df_valor['Age'] == edad
  evaluacion_correlacion(df_valor.loc[filtro_edad, 'Overall'], 
                         'Nivel para edad ' +str(edad), 
                         df_valor.loc[filtro_edad, 'Value_€'], 
                         'Valor para edad ' +str(edad))

# 05 Ajuste por collider
Ajustar a (quedarnos con ciertos valores de) un collider (causa común a dos variables) puede modificar la asociación que vemos entre esas dos variables.

## EJERCICIO 2
Queremos entender la influencia de la edad sobre el nivel futbolístico.

Link al diagrama causal: https://docs.google.com/presentation/d/1jYSOQpCVbL7JdacYhq8NuKWwhYe2QwW8qvM680GERps/edit#slide=id.gbc327d14bd_1_6

Primero, visualiza la relación entre ambas.

In [ ]:
evaluacion_correlacion(df_valor['Age'], 'Edad', 
                       df_valor['Overall'], 'Nivel')

Ahora visualiza la misma relación entre variables pero sólo de aquellos jugadores con un valor mediano. Para esto, calcula el percentil 44 y 55 de la columna valor y crea un filtro que se quede con solo con los jugadores cuyo valor este entre ambos.

In [ ]:
# Generamos el filtro
valor_min = df_valor['Value_€'].quantile(0.45)
valor_max = df_valor['Value_€'].quantile(0.55)

filtro_medianos = ((df_valor['Value_€'] >= valor_min)&
                   (df_valor['Value_€'] <= valor_max))

In [ ]:
evaluacion_correlacion(df_valor.loc[filtro_medianos,'Age'], 'Edad', 
                       df_valor.loc[filtro_medianos, 'Overall'], 'Nivel')

## Por qué varía tanto, que explicación tiene?